In [26]:
from bs4 import BeautifulSoup
import urllib
import requests
from IPython.core.display import display,HTML
import numpy as np
import pandas as pd
import re

In [27]:
#functions for populating feature lists

def movdet(cast): #gets actor and director names
    urlmain = 'https://www.the-numbers.com'
    urlmov=urlmain+cast
    #Request HTML and parse
    response = requests.get(urlmov)
    page = response.text
    soup = BeautifulSoup(page,"lxml")

    actor= list()
    try:
        for i in soup.find_all(itemprop = "actor"):
            for a in i.find_all('a', href=True):
                actor.append(a['href'])
        actor2= list()
        for i in range(len(actor)):
            a, b= str(actor[i]).split("-", 1)
            actor2.append(b)
        star.append(actor2[0])
        director1= list()
        for i in soup.find_all(itemprop = "director"):
            for a in i.find_all('a', href=True):
                director1.append(a['href'])
        director2= list()
        for i in range(len(director1)):
            a, b= str(director1[i]).split("-", 1)
            director2.append(b)
        director.append(director2[0])
    except:
        star.append("None")
        director.append("None")
    

def movsum(sumlist): # gets mpaa rating and genre
    tempurl="https://www.the-numbers.com"+str(sumlist)
    tempres=requests.get(tempurl)
    temppage = tempres.text
    tempsoup = BeautifulSoup(temppage,"lxml")

    mpaa=tempsoup.find("a", href=re.compile("market/mpaa-rating"))
    mpaa= str(mpaa).strip('<a href="/market/mpaa-rating/').split("-(US)", 1)
    rating.append(mpaa[0])

    gen=tempsoup.find("a", href=re.compile("/market/genre/"))
    gen= str(gen).strip('<a href="/market/genre/').split('">', 1)
    genre.append(gen[0])

In [28]:
### Scraping names of films
#this list will be used to later used to scrape page of each film individually 

In [101]:
#getting inital data from html
    
url1 = 'https://www.the-numbers.com/movie/budgets/all/901'


response = requests.get(url1)
response.status_code 
myhtml= response.text

soup = BeautifulSoup(myhtml, 'lxml')

In [102]:
#lists for  features i actually was able to scrape
rank=[]
budget=[]
dom_gross=[]
ww_gross=[]
star=list()
costar=list()
director=list()
producer=list()
rating=list()
genre=list()


In [103]:
## Data cleaning

In [104]:
#cleaning data to retain only numeric values
budgets=[]
for elem in soup.find_all(class_='data'):
    budgets.append(elem.text)

for i in range(len(budgets)):
    budgets[i]=(re.sub("[^0-9]", "",  budgets[i]))
budgets= list(budgets)

 # rank number 
for i in range(0, len(budgets), 4):
    rank.append(budgets[i])

# budget
for i in range(1, len(budgets), 4):
    budget.append(budgets[i])

# domestic gross
for i in range(2, len(budgets), 4):
    dom_gross.append(budgets[i])

 # worldwide gross
for i in range(3, len(budgets), 4):
    ww_gross.append(budgets[i])


In [105]:
#lists for storing links to each film 
link_lst= list() #temp list of all links in table
sumlist=list() #list of links to movie summary tab
cast = list()  #list of links to movie cast&crew tab

table = soup.find('table')
 
for a in table.find_all('a', href=True):
    link_lst.append(a['href'])

dates= list()
for i in range(0, len(link_lst), 2):
    link_lst[i]= link_lst[i].replace('/box-office-chart/daily/', '')
    dates.append(link_lst[i])
    

for i in range(1, len(link_lst), 2):
    sumlist.append(link_lst[i])
    link_lst[i]= link_lst[i].replace('=summary', '=cast-and-crew')
    cast.append(link_lst[i])


In [106]:
#scraping each mov page 
for i in range(len(cast)):
    movdet(cast[i])
for j in range(len(sumlist)):
    movsum(sumlist[i])

In [107]:
# verifying all my lists have same number of elements
print('cast={}'.format(len(cast)))
print('sumlist={}'.format(len(sumlist)))
print('dir={}'.format(len(director)))
print('star={}'.format(len(star)))
print('gen={}'.format(len(genre)))
print('rat={}'.format(len(rating)))

cast=100
sumlist=100
dir=100
star=100
gen=100
rat=100


In [108]:
#creates pandas df from all lists
df=pd.DataFrame(list(zip(rank, budget, dom_gross, ww_gross, director, star, dates, genre, rating)), 
               columns =['Number', 'Budget', 'Domestic Gross', 'Worldwide Gross', 'Director', 'Main Actor', 'Release Date', 'Genre', 'MPAA'])
print(df.head)

<bound method NDFrame.head of    Number    Budget Domestic Gross Worldwide Gross          Director  \
0     901  60000000       90580000       165890634     Chuck-Russell   
1     902  60000000       90380162       196439693  Robert-Schwentke   
2     903  60000000       88200225       170200225     Todd-Phillips   
3     904  60000000       82670733       157670733        Tony-Scott   
4     905  60000000       82571173       135015330              None   
..    ...       ...            ...             ...               ...   
95    996  59000000       16790790        28296983       Joe-Cornish   
96    997  58000000      363070709       784682527        Tim-Miller   
97    998  58000000      350126372       547326372    Clint-Eastwood   
98    999  58000000       80069458       158893300     Rob-Letterman   
99   1000  58000000       53715611       188715611              None   

                Main Actor Release Date   Genre MPAA  
0           Dwayne-Johnson   2002/04/18  Action   

In [109]:
#makes csv file from df

df.to_csv(r'C:\Users\TCIDEMIRKAN\Desktop\DS BootCamp\DSBootCampNov\curriculum\project-02\project-02-introduction\CSV\movdf10.csv', index= False)